In [13]:
!java -mx6g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -timeout 5000

^C


https://towardsdatascience.com/natural-language-processing-using-stanfords-corenlp-d9e64c1e1024

In [91]:
result = nlp.annotate(text,
                       properties={
                           'annotators': 'sentiment',
                           'outputFormat': 'json',
                           'timeout': 1000,
                       })
x = result['sentences'][0]['sentimentTree']
x = re.sub('\\r\\n', '', x)
x = re.sub('\s+', ' ', x)

groups = [x]

for group in groups:
    #  раскрываем внешние скобки
    group = group[1:len(group)-1]
    #  убираем мусор
    group = re.findall('\(.+\)', group)
    if len(group) == 0:
        continue
    elif len(group) > 1:
        print('Regrex error')
        break
    else:
        group = group[0]
    #  добавляем в список все отдельно стоящие пары скобок
    counter = 0
    start_char = 0
    for num, char in enumerate(group):
        if char == '(':
            counter += 1
        elif char == ')':
            counter -= 1
        potential_group = group[start_char: num + 1]
        potential_group = potential_group.strip(' ')
        if (counter == 0)and(potential_group.startswith('(')):
            groups.append(potential_group)
            start_char = num + 1

In [99]:
groups[0]

'(ROOT|sentiment=3|prob=0,503 (@S|sentiment=3|prob=0,500 (@S|sentiment=3|prob=0,492 (S|sentiment=3|prob=0,457 (NP|sentiment=2|prob=0,999 We) (VP|sentiment=3|prob=0,463 (@VP|sentiment=2|prob=0,675 (VP|sentiment=2|prob=0,806 (VBD|sentiment=2|prob=0,996 arrived) (PP|sentiment=2|prob=0,798 (IN|sentiment=2|prob=1,000 on) (NP|sentiment=2|prob=0,824 (DT|sentiment=2|prob=0,998 an) (@NP|sentiment=2|prob=0,718 (JJ|sentiment=2|prob=0,996 early) (NN|sentiment=2|prob=0,987 flight))))) (CC|sentiment=2|prob=0,998 and)) (VP|sentiment=2|prob=0,514 (VBD|sentiment=2|prob=0,998 wanted) (S|sentiment=2|prob=0,498 (TO|sentiment=2|prob=0,999 to) (VP|sentiment=2|prob=0,514 (@VP|sentiment=3|prob=0,586 (VB|sentiment=2|prob=0,982 drop) (NP|sentiment=3|prob=0,947 (PRP$|sentiment=2|prob=0,996 our) (NNS|sentiment=2|prob=0,370 bags))) (PP|sentiment=2|prob=0,998 (IN|sentiment=2|prob=0,997 at) (NP|sentiment=2|prob=1,000 (DT|sentiment=2|prob=0,999 the) (NN|sentiment=2|prob=0,989 hotel)))))))) (:|sentiment=2|prob=0,999 -

In [103]:
group = re.sub('\(.+?\|sentiment=\d\|prob=\d,\d\d\d |\)', ' ', groups[0])
group = re.sub('\s+', ' ', group)
group

' We arrived on an early flight and wanted to drop our bags at the hotel -- they were able to check us in at 11:00 am . '

In [106]:
pos = []
neg = []
for group in groups:
    sentiment = re.findall('^\(.+?\|sentiment=(\d)\|', group)[0]
    sentiment = int(sentiment)
    synt = re.findall('^\((.+?)\|sentiment=\d\|', group)[0]
    print(synt)
    if sentiment > 2:
        pos.append(group)
    elif sentiment == 1:
        neg.append(group)
    

ROOT
@S
.
@S
S
S
:
NP
VP
NP
VP
VBD
ADJP
@VP
VP
JJ
S
VP
CC
VBD
S
TO
VP
VBD
PP
TO
VP
@VP
PP
IN
NP
@VP
PP
VB
NP
IN
PP
DT
@NP
VB
NP
IN
NP
IN
NP
JJ
NN
PRP$
NNS
DT
NN
CD
NN


In [14]:
from pycorenlp import StanfordCoreNLP
import pandas as pd
import re
from tqdm import tqdm
from nltk.tokenize import sent_tokenize

In [16]:
reviews = pd.read_csv('reviews.csv', sep='\t')

In [17]:
ids = reviews['id'].unique().tolist()

In [18]:
len(ids)

2197

In [19]:
nlp = StanfordCoreNLP('http://localhost:9000')

In [184]:
texts = reviews[reviews['id'] == 'AV--lQdFRxPSIh2RmtbT']['reviews.text'].tolist()

In [185]:
texts

["I read all the ratings before I booked the Best Western Mission Bay. Couldn't believe it could be as good as I was reading for the price I was given. Well it is everything and more. It is a 3 floor hotel with elevator and easy to climb stairs. The room was just beautiful, as nice as a 5 star hotel I had previously stayed in. They offer a free breakfast with juic3, coffee, eggs, bacon, waffles, donuts, toast, danish and more. I couldn't believe how much when I walked in. The room was a bit small but people were nice with making room. This hotel is so close to Old Town and Sea World you never need to get back on the freeway unless you chooose to. I went up to Mission Bay and followed that into Old Town. Rooms are a great size also and lots of parking available. Its a fantastic hotel to choose and I will stay here on my next San Diego visit. The staff was the friendliest people I have ever met. The man who checked me in made sure I was happy and under stood everything. Pool has a code a

In [181]:
texts = [sent_tokenize(text) for text in texts]
texts_splitted = []
for text in texts:
    while len(text) > 5:
        texts_slitted.append(text[:5])
        text = text[5:]
texts = [' '.join(text) for text in texts]

NameError: name 'texts_slitted' is not defined

In [168]:
pos = []
neg = []
for text in tqdm(texts):
    result = nlp.annotate(text,
                       properties={
                           'annotators': 'sentiment',
                           'outputFormat': 'json',
                           'timeout': 1000,
                       })
    for sent in result['sentences']:
        tree = sent['sentimentTree']
        tree_split = tree.split('\r\n')
        tree_split = [i.strip(' ') for i in tree_split]
        NPs = []
        for num, i in enumerate(tree_split):
            if i.startswith('(@NP'):
                sample = ' '.join(tree_split[num:])
                count = 0
                for numb, char in enumerate(sample):
                    if char == '(':
                        count += 1
                    elif char == ')':
                        count -= 1
                    if count == 0:
                        NPs.append(sample[:numb + 1])
                        break
        for NP in NPs:
            sentiment = re.findall('^\(@NP\|sentiment=(\d)\|', NP)
            sentiment = int(sentiment[0])
            keyword = re.findall('\([^()]+?\|sentiment=\d\|prob=[^()]+?\s([^()]+?)\)', NP)
            if sentiment > 2:
                pos.append(' '.join(keyword))
            elif sentiment == 1:
                neg.append(' '.join(keyword))

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00,  6.84it/s]


In [169]:
pos

['the marina , good food and',
 'the marina , good food',
 'terrific view',
 'good restaurants',
 'Starbucks right',
 'pleasure having you with us']

In [170]:
neg

[]

In [132]:
tree = result['sentences'][4]['sentimentTree']

In [133]:
tree_split = tree.split('\r\n')
tree_split = [i.strip(' ') for i in tree_split]
NPs = []
for num, i in enumerate(tree_split):
    if i.startswith('(@NP'):
        sample = ' '.join(tree_split[num:])
        count = 0
        for numb, char in enumerate(sample):
            if char == '(':
                count += 1
            elif char == ')':
                count -= 1
            if count == 0:
                NPs.append(sample[:numb + 1])
                break

In [134]:
pos = []
neg = []
for NP in NPs:
    sentiment = re.findall('^\(@NP\|sentiment=(\d)\|', NP)
    sentiment = int(sentiment[0])
    text = re.findall('\([^()]+?\|sentiment=\d\|prob=[^()]+?\s([^()]+?)\)', NP)
    print(text)

In [201]:
import pytreebank

In [207]:
dataset

In [210]:
example.to_labeled_lines()

[(None,
  'NP|sentiment=2|prob=0,999 We VBD|sentiment=2|prob=0,996 arrived IN|sentiment=2|prob=1,000 on DT|sentiment=2|prob=0,998 an JJ|sentiment=2|prob=0,996 early NN|sentiment=2|prob=0,987 flight CC|sentiment=2|prob=0,998 and VBD|sentiment=2|prob=0,998 wanted TO|sentiment=2|prob=0,999 to VB|sentiment=2|prob=0,982 drop PRP$|sentiment=2|prob=0,996 our NNS|sentiment=2|prob=0,370 bags IN|sentiment=2|prob=0,997 at DT|sentiment=2|prob=0,999 the NN|sentiment=2|prob=0,989 hotel :|sentiment=2|prob=0,999 -- NP|sentiment=2|prob=0,999 they VBD|sentiment=2|prob=0,996 were JJ|sentiment=3|prob=0,987 able TO|sentiment=2|prob=0,999 to VB|sentiment=3|prob=0,833 check NP|sentiment=3|prob=0,979 us IN|sentiment=2|prob=0,999 in IN|sentiment=2|prob=0,997 at CD|sentiment=2|prob=0,625 11:00 NN|sentiment=2|prob=0,997 am .|sentiment=2|prob=1,000 .'),
 (None,
  'NP|sentiment=2|prob=0,999 We VBD|sentiment=2|prob=0,996 arrived IN|sentiment=2|prob=1,000 on DT|sentiment=2|prob=0,998 an JJ|sentiment=2|prob=0,996 ear

In [208]:
dataset = pytreebank.import_tree_corpus("test.txt")
example = dataset[0]

# extract spans from the tree.
for label, sentence in example.to_lines():
	print("%s has sentiment label %s" % (
		sentence,
		["very negative", "negative", "neutral", "positive", "very positive"][label]
	))

TypeError: list indices must be integers or slices, not NoneType

In [223]:
text = 'We arrived on an early flight and wanted to drop our bags at the hotel--they were able to check us in at 11:00am. That was a great start to our stay. The room and breakfast were standard Best Western fare. The location was perfect for our three days in San Diego. The hotel is just off I-5 and less than 20 minutes to Coronado, La Jolla, Old Town, etc. I have to mention Irma who ran the breakfast room. She was pleasant, kept things full, and checked in with every guest. Great service!'
result = nlp.annotate(text,
                       properties={
                           'annotators': 'sentiment',
                           'outputFormat': 'json',
                           'timeout': 1000,
                       })

In [224]:
print(result)

{'sentences': [{'index': 0, 'parse': '(ROOT\r\n  (S\r\n    (S\r\n      (NP (PRP We))\r\n      (VP\r\n        (VP (VBD arrived)\r\n          (PP (IN on)\r\n            (NP (DT an) (JJ early) (NN flight))))\r\n        (CC and)\r\n        (VP (VBD wanted)\r\n          (S\r\n            (VP (TO to)\r\n              (VP (VB drop)\r\n                (NP (PRP$ our) (NNS bags))\r\n                (PP (IN at)\r\n                  (NP (DT the) (NN hotel)))))))))\r\n    (: --)\r\n    (S\r\n      (NP (PRP they))\r\n      (VP (VBD were)\r\n        (ADJP (JJ able)\r\n          (S\r\n            (VP (TO to)\r\n              (VP (VB check)\r\n                (NP (PRP us))\r\n                (PP (IN in)\r\n                  (PP (IN at)\r\n                    (NP (CD 11:00) (NN am))))))))))\r\n    (. .)))', 'binaryParse': '(ROOT\r\n  (S\r\n    (@S\r\n      (@S\r\n        (S\r\n          (NP (PRP We))\r\n          (VP\r\n            (@VP\r\n              (VP (VBD arrived)\r\n                (PP (IN on)\r

In [228]:
pytreebank.create_tree_from_string(result['sentences'][0]['sentimentTree']).to_lines()

['NP|sentiment=2|prob=0,999 We VBD|sentiment=2|prob=0,996 arrived IN|sentiment=2|prob=1,000 on DT|sentiment=2|prob=0,998 an JJ|sentiment=2|prob=0,996 early NN|sentiment=2|prob=0,987 flight CC|sentiment=2|prob=0,998 and VBD|sentiment=2|prob=0,998 wanted TO|sentiment=2|prob=0,999 to VB|sentiment=2|prob=0,982 drop PRP$|sentiment=2|prob=0,996 our NNS|sentiment=2|prob=0,370 bags IN|sentiment=2|prob=0,997 at DT|sentiment=2|prob=0,999 the NN|sentiment=2|prob=0,989 hotel :|sentiment=2|prob=0,999 -- NP|sentiment=2|prob=0,999 they VBD|sentiment=2|prob=0,996 were JJ|sentiment=3|prob=0,987 able TO|sentiment=2|prob=0,999 to VB|sentiment=3|prob=0,833 check NP|sentiment=3|prob=0,979 us IN|sentiment=2|prob=0,999 in IN|sentiment=2|prob=0,997 at CD|sentiment=2|prob=0,625 11:00 NN|sentiment=2|prob=0,997 am .|sentiment=2|prob=1,000 .',
 'NP|sentiment=2|prob=0,999 We VBD|sentiment=2|prob=0,996 arrived IN|sentiment=2|prob=1,000 on DT|sentiment=2|prob=0,998 an JJ|sentiment=2|prob=0,996 early NN|sentiment=2|p